**Imports**

In [1]:
# imports
import os
import json
import numpy as np

# Import torch
import torch
import torch.nn as nn
import torch.optim as optim

**Sample data**

In [2]:
path_dir = os.path.dirname("")
with open(os.path.join(path_dir, 'timeseries_pheno_metrics.json')) as f:
    json = json.load(f)

X_train = json["timeseries_pheno_metrics"]
y_train = json["label_id"]

**Convert data to PyTorch tensors**

In [3]:
X_train_tensor = torch.tensor(np.expand_dims(X_train, axis=1), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

print(X_train_tensor.shape)

torch.Size([862, 1, 41])


**LSTM classifier model**

In [4]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

**Define model parameters**

In [5]:
input_size = X_train_tensor.shape[2]
hidden_size = 64
num_layers = 2
output_size = len(set(y_train_tensor))

**Instantiate**

In [6]:
model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)

**Loss function and optimizer**

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

**Train the model**

In [8]:
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/1000], Loss: 6.779297351837158
Epoch [2/1000], Loss: 6.767672538757324
Epoch [3/1000], Loss: 6.75636625289917
Epoch [4/1000], Loss: 6.7452168464660645
Epoch [5/1000], Loss: 6.733717441558838
Epoch [6/1000], Loss: 6.721713066101074
Epoch [7/1000], Loss: 6.709072589874268
Epoch [8/1000], Loss: 6.696264743804932
Epoch [9/1000], Loss: 6.682994842529297
Epoch [10/1000], Loss: 6.66933012008667
Epoch [11/1000], Loss: 6.655234336853027
Epoch [12/1000], Loss: 6.640329360961914
Epoch [13/1000], Loss: 6.624411106109619
Epoch [14/1000], Loss: 6.607630252838135
Epoch [15/1000], Loss: 6.590010643005371
Epoch [16/1000], Loss: 6.571131229400635
Epoch [17/1000], Loss: 6.551170825958252
Epoch [18/1000], Loss: 6.530298233032227
Epoch [19/1000], Loss: 6.508384704589844
Epoch [20/1000], Loss: 6.485522747039795
Epoch [21/1000], Loss: 6.46098518371582
Epoch [22/1000], Loss: 6.435241222381592
Epoch [23/1000], Loss: 6.407843589782715
Epoch [24/1000], Loss: 6.3788065910339355
Epoch [25/1000], Loss: 6.3

**Making prediction**

In [9]:
from cshd import cube_query, get_timeseries, params_phenometrics, calc_phenometrics, cshd_array

In [10]:
def return_class_by_id(id):
    if 370: return "Soja"
    if 372: "Arroz"
    if 359: "Vegetação Florestal"
    if 365: "Corpos d'agua"
    if 367: return "Superfícies Artificiais"
    if 363: return "Pastagem"
    if 361: return "Formação Campestre"

In [11]:
S2_cube = cube_query(
    collection="S2-16D-2",
    start_date="2023-01-01",
    end_date="2023-12-31",
    freq='16D',
    band="NDVI"
)

config = params_phenometrics(
    peak_metric='pos', 
    base_metric='vos', 
    method='seasonal_amplitude', 
    factor=0.2, 
    thresh_sides='two_sided', 
    abs_value=0.1
)

longitude = -52.97612606396396
latitude =  -29.325361867915753

ts = get_timeseries(
    cube=S2_cube, 
    geom=[dict(coordinates = [longitude, latitude])],
    cloud_filter = True
)
ndvi_array = cshd_array(
    timeserie=ts['values'],
    start_date='2023-01-01',
    freq='16D'
)
ds_phenos = calc_phenometrics(
    da=ndvi_array,
    engine='phenolopy',
    config=config,
    start_date='2023-01-01'
)
test_tl = ts['values'] + ds_phenos

Initialising calculation of phenometrics.

Beginning extraction of CRS metadata.
> Extracting CRS metadata.
> No CRS metadata found. Returning None.

Beginning calculation of phenometrics. This can take awhile - please wait.

Beginning calculation of peak of season (pos) values and times.
> Calculating peak of season (pos) values.
> Calculating peak of season (pos) times.
> Success!

Beginning calculation of valley of season (vos) values and times.
> Calculating valley of season (vos) values.
> Calculating valley of season (vos) times.
> Success!

Beginning calculation of middle of season (mos) values (times not possible).
> Calculating middle of season (mos) values.
> Success!

Beginning calculation of base (bse) values (times not possible).
> Calculating base (bse) values.
> Success!

Beginning calculation of amplitude of season (aos) values (times not possible).
> Calculating amplitude of season (aos) values.
> Success!

Beginning calculation of start of season (sos) values and time

In [12]:
X_test_tensor = torch.tensor(np.expand_dims([test_tl], axis=1), dtype=torch.float32)
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_labels = torch.argmax(predictions, dim=1)
    print("Predicted Label:", predicted_labels[0], return_class_by_id(predicted_labels[0]))

Predicted Label: tensor(370) Soja
